# Finding My Vacation

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store city data into DataFrame

In [ ]:
cities_data = pd.read_csv("output_data/cities.csv")
cities_data.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [ ]:
# Set up locations and weight for the map
locations = cities_data[['Lat', 'Lng']]
weights = cities_data['Humidity']

# Display heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
ideal_cities = cities_data.loc[(cities_data['Max Temp'] <= 24.4) & (cities_data['Humidity'] <= 68) & (cities_data['Wind Speed'] <= 2) & (cities_data['Cloudiness'] <= 0)]
ideal_cities

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Create the hotels dataframe with a column for Hotel Name
hotel_df = ideal_cities[['City', 'Country', 'Lat', 'Lng']]
hotel_df

In [ ]:
# Set up the base url to query a nearby search in google API
nearby_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through the cities dataframe, using the latitude and longitude data for the location parameter in the API request.
# Store the hotel name, latitude and longitude in lists. Use the latitude and longitude from the query to 
# perform another query for the hotel's city and country. Store each in separate lists
hotel_names = []
for index, row in hotel_df.iterrows():
    latitude = row[2]
    longitude = row[3]
    hotel_query_url = f"{nearby_url}?keyword=hotel&location={latitude}%2C{longitude}&radius=5000&type=hotel&key={g_key}"
    hotel_response = requests.get(hotel_query_url).json()
    try:
        hotel_names.append(hotel_response['results'][0]['name'])
    except:
        hotel_names.append('No hotel within 5000 meters!')

# Use the lists to fill in the hotels dataframe
hotel_df['Hotel Name'] = hotel_names
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='black',
    stroke_color='purple', scale=5,
    info_box_content=hotel_info
)
# Display figure
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig